In [1]:
import json, csv
from nltk.corpus import wordnet as wn
import networkx as nx

# Build the legit-synset graph
G = nx.DiGraph()
G.add_nodes_from(x.name() for x in wn.all_synsets() if x.pos() == "n")
for child in wn.all_synsets():
    if child.pos() != "n":
        continue
    
    for parent in child.hypernyms():
        if child.pos() != "n":
            continue
        G.add_edge(parent.name(), child.name())
        
# Add the illegit-synset graph
with open(r"C:\Users\cgokmen\research\bddl\utils\custom_synsets.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        child = row["custom_synset"].strip()
        parent = wn.synset(row["hypernyms"].strip()).name()
        assert parent in G.nodes, "Could not find " + parent
        G.add_edge(parent, child)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\cgokmen\\research\\bddl\\utils\\custom_synsets.csv'

In [10]:
# Get all the category mappings
import collections

categories = []
synset_to_cat_and_mass = collections.defaultdict(dict)
with open(r"D:\ig_pipeline\metadata\category_mapping.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        s = row["synset"].strip()
        try:
            s = wn.synset(s).name()
        except:
            pass
        c = row["category"].strip()
        mass_str = row["mass"].strip()
        m = float(mass_str) if mass_str else None
        if s not in G.nodes:
            print("Could not find ", s, c)
            # continue
        synset_to_cat_and_mass[s][c] = m
        categories.append((c, s))

Could not find  olive_oil__bottle.n.01 olive_oil_bottle
Could not find  water__bottle.n.06 water_bottle
Could not find  knife_block.n.01 knife_block
Could not find  dehumidifier.n.01 dehumidifier
Could not find  dreidel.n.01 dreidel
Could not find  hamster_wheel.n.01 hamster_wheel
Could not find  blueberry_mousse.n.01 blueberry_mousse
Could not find  insect_repellent__bottle.n.01 bug_repellent_bottle
Could not find  liquid_soap__bottle.n.01 dish_soap_bottle
Could not find  goalkeeper_gloves.n.01 goalkeeper_gloves
Could not find  kielbasa.n.01 kielbasa
Could not find  litter_box.n.01 litter_boxh
Could not find  pet_bed.n.01 pet_bed
Could not find  pizza_box.n.01 pizza_box
Could not find  pumpkin_pie_spice.n.01 pumpkin_pie_spice
Could not find  tupperware.n.01 tupperware
Could not find  vodka__bottle.n.01 vodka_bottle
Could not find  whitewall_tire.n.01 whitewall_tire
Could not find  lemon_juice__bottle.n.01 lemon_juice_bottle
Could not find  liquid_soap__bottle.n.01 liquid_soap_bottle
C

In [3]:
# Add a list of masses to every node, going from bottom to top
topological_order = list(reversed(list(nx.topological_sort(G))))
for node in topological_order:
    # Get this node's immediate children. Their stuff has already been collected
    this_cat_and_mass = {}
    for succ in G.successors(node):
        this_cat_and_mass.update(G.nodes[succ]["cat_and_mass"])
        
    # Add the stuff that belongs to this particular node
    this_cat_and_mass.update(synset_to_cat_and_mass[node])
    G.nodes[node]["cat_and_mass"] = this_cat_and_mass

In [4]:
import numpy as np
# Get the mean mass for each node
for node in G.nodes:
    this_cat_and_mass = G.nodes[node]["cat_and_mass"]
    all_masses = [mass for mass in this_cat_and_mass.values() if mass is not None]
    if all_masses:
        G.nodes[node]["mean_mass"] = np.mean(all_masses) if all_masses else None
        # G.nodes[node]["estimation_distance"] = 0

In [10]:
# Propagate the masses down now.
reverse_topological_order = list(nx.topological_sort(G))
for node in reverse_topological_order:
    if "mean_mass" in G.nodes[node]:
        continue
    parents = list(G.predecessors(node))
    parent_masses = [G.nodes[p]["mean_mass"] for p in parents if "mean_mass" in G.nodes[p]]
    if not parent_masses:
        continue
    G.nodes[node]["mean_mass"] = np.mean(parent_masses)
    # G.nodes[node]["estimation_distance"] = G.nodes[parent]["estimation_distance"] + 1

In [11]:
for c, s in categories:
    mass = G.nodes[s]["mean_mass"]
    print(mass)

75.4087
10.03970227975
75.4087
21.13058332
1.0886208
57.606184
57.12537648
13.89805888
22.440708213333334
22.440708213333334
18.14368
16.782904
2.4040376
9.07184
7.257472
3.5380176
11.61999051
11.793392
221.1261
20.638436
18.14368
30.390664
26.308336
70.5193702
4.29551624
9.07184
92.98636
18.14368
21.318824
1.7463292
3.96893
9.07184
1.7123098
6.5317248
12.0315278
4.51097244
3.259997455
63.50288
18.14368
228.156776
0.47173568
1.13398
74.729282
4.53592
4.79900336
13.50343384
9.07184
11.0903244
236.321432
9.4573932
41.59892232
6.5317248
18.14368
17.5540104
6.16658324
77.564232
9.07184
12.5191392
0.80739376
33.8152836
22.440708213333334
0.907184
3.175144
73.481904
0.57152592
17.5540104
72.99994039
29.256684
0.0675
0.068
0.01
0.12
0.0001
0.1
0.2517437654
0.1
0.011
3.0
0.02
5.0
0.465
0.2
0.8
2.0
0.5
0.06
0.3061748498
1.787543976
0.35
0.024
0.0075
0.226796
0.015
1.451495584
0.02
3.871410878
0.5249999999999999
0.771107029
0.55
1.5
0.2
0.3855535145
1.675
0.3
0.15
4.0
0.765437
0.4510891
0.03
0.5

In [2]:
with open(r"D:\ig_pipeline\artifacts\pipeline\object_inventory.json", "r") as f:
    inv = json.load(f)

In [3]:
provided_cats = {x.split("-")[0] for x in inv["providers"].keys()}

In [11]:
for cat, _ in categories:
    print(int(cat in provided_cats))

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [9]:
len(categories)

1426